# Import Libs

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
# import altair as alt
import matplotlib.pyplot as plt
import warnings
import folium
warnings.filterwarnings('ignore')

In [ ]:
file = '../data/raw/CriticalPath_Data_EM_Confidential.xlsx'

data = pd.read_excel(file)

# Create ClusterMap of Data

In [ ]:
M = data.isnull().astype(int)
cols = M.std()==0
shadow_matrix = M[list(M.columns[~cols])]

clustergrid = sns.clustermap(shadow_matrix.corr(),figsize=(12,12),cmap='coolwarm');

# Evaluate Wages v. Test Scores

## Parental Wages

In [ ]:
for col in ['Father_Wages','Mther_Wages','Net_worth_parents_investments','Parent_income_AGI','Parent_cash','Net_worth_parents_bus']:

  sns.lmplot(fit_reg=True,data=data[data['Enrolled']==80],y=col,x='MAXSATVerbalMath',size=5)
  plt.ylim(np.nanpercentile(data[col],2.5),
            np.nanpercentile(data[col],97.5))
  
  sns.lmplot(fit_reg=True,data=data[data['Enrolled']==80],y=col,x='ACTComposite',size=5)
  plt.ylim(np.nanpercentile(data[col],2.5),
            np.nanpercentile(data[col],97.5))

## Student Wages

In [ ]:
for col in ['Student_Wages','Student_income_AGI','Net_worth_students_investments']:

  sns.lmplot(fit_reg=True,data=data[data['Enrolled']==80],y=col,x='MAXSATVerbalMath',size=5)
  plt.ylim(np.nanpercentile(data[col],2.5),
            np.nanpercentile(data[col],97.5))
  
  sns.lmplot(fit_reg=True,data=data[data['Enrolled']==80],y=col,x='ACTComposite',size=5)
  plt.ylim(np.nanpercentile(data[col],2.5),
            np.nanpercentile(data[col],97.5))

# Where else do People go???

In [ ]:
data[~data['College_chosen_by_non-matrics'].isnull()].size/len(data.columns)

In [ ]:
data['College_chosen_by_non-matrics'].max()

# Import [CEEB Codes](https://help.liaisonedu.com/WebAdMIT_Help_Center/Documents_and_Reference_Guides/Master_College_Code_List)

How many students report this number???

In [ ]:
len(data[~data['College_chosen_by_non-matrics'].isnull()])

In [ ]:
college_codes = path + 'collegeCEEBS.xlsx'
college_codes = pd.read_excel(college_codes)

In [ ]:
mapper = college_codes[~college_codes['CEEB Code'].isnull()][['Name']]
mapper.index = college_codes[~college_codes['CEEB Code'].isnull()].index = college_codes[~college_codes['CEEB Code'].isnull()]['CEEB Code'].astype(float)
mapper = mapper.to_dict()

In [ ]:
colleges_by_major = data[['College_chosen_by_non-matrics','Major','Unique_student_ID']]
colleges_by_major['College_chosen_by_non-matrics'] = colleges_by_major['College_chosen_by_non-matrics'].map(mapper['Name'])
colleges_by_major = colleges_by_major[~colleges_by_major['College_chosen_by_non-matrics'].isnull()]

In [ ]:
college_by_major = colleges_by_major.groupby(["College_chosen_by_non-matrics",
                                              "Major"]).count().rename(columns={"Unique_student_ID":"Num Students"})
college_by_major = college_by_major.reset_index()
college_by_major[college_by_major['College_chosen_by_non-matrics']=='UNIVERSITY AT ALBANY']

In [ ]:
alt.Chart(college_by_major[college_by_major['College_chosen_by_non-matrics']=='UNIVERSITY AT ALBANY']).mark_bar().encode(
    x='Num Students:Q',
    y=alt.Y(
        'Major:O',
        sort = alt.EncodingSortField(
                field='Num Students',
                op = "sum",
                order = "descending"
        )
    )
).properties(height=400)

In [ ]:
len(data[~data['College_chosen_by_non-matrics'].isnull()])